In [2]:
import polars as pl

In [3]:
df_datos = pl.read_csv("../train.csv")

In [26]:
pl.Config.set_tbl_rows(10)

polars.config.Config

In [41]:
## Calcula la media de conteo de glóbulos blancos y rojos por desorden genético para los pacientes que tienen menos de 5 años, y posteriomente calcula estos valores para los pacientes que tienen 5 años o más. Junta ambos resultados en un solo DataFrame y calcula la diferencia absoluta de ambas métricas entre ambos grupos de edad.

In [16]:

df_subclass_minors5 = (
    df_datos.filter(pl.col("Patient Age") < 5).group_by("Disorder Subclass")
    .agg(
        mean_red_m = pl.col("Blood cell count (mcL)").mean(),
        mean_white_m = pl.col("White Blood cell count (thousand per microliter)").mean(),
    )
    .drop_nulls()
)
df_subclass_minors5

Disorder Subclass,mean_red_m,mean_white_m
str,f64,f64
"""Cancer""",4.883708,7.661741
"""Leigh syndrome""",4.896911,7.419381
"""Diabetes""",4.887763,7.595284
"""Leber's hereditary optic neuro…",4.883083,7.529758
"""Tay-Sachs""",4.900089,7.647843
"""Mitochondrial myopathy""",4.893421,7.462122
"""Hemochromatosis""",4.889836,7.454434
"""Alzheimer's""",4.896252,7.740429
"""Cystic fibrosis""",4.898628,7.630248


In [17]:
df_subclass_higher5 = (
    df_datos.filter(pl.col("Patient Age") >= 5).group_by("Disorder Subclass")
    .agg(
        mean_red_h = pl.col("Blood cell count (mcL)").mean(),
        mean_white_h = pl.col("White Blood cell count (thousand per microliter)").mean(),
    )
    .drop_nulls()
)
df_subclass_higher5

Disorder Subclass,mean_red_h,mean_white_h
str,f64,f64
"""Hemochromatosis""",4.892835,7.457545
"""Cancer""",4.851911,7.765564
"""Mitochondrial myopathy""",4.897351,7.519142
"""Cystic fibrosis""",4.902574,7.468543
"""Leigh syndrome""",4.905123,7.450601
"""Alzheimer's""",4.897511,7.247322
"""Diabetes""",4.896993,7.37063
"""Leber's hereditary optic neuro…",4.907231,7.474441
"""Tay-Sachs""",4.902508,7.416985


In [18]:
df_inner = df_subclass_minors5.join(df_subclass_higher5, on="Disorder Subclass",how="inner")
df_inner

Disorder Subclass,mean_red_m,mean_white_m,mean_red_h,mean_white_h
str,f64,f64,f64,f64
"""Hemochromatosis""",4.889836,7.454434,4.892835,7.457545
"""Cancer""",4.883708,7.661741,4.851911,7.765564
"""Mitochondrial myopathy""",4.893421,7.462122,4.897351,7.519142
"""Cystic fibrosis""",4.898628,7.630248,4.902574,7.468543
"""Leigh syndrome""",4.896911,7.419381,4.905123,7.450601
"""Alzheimer's""",4.896252,7.740429,4.897511,7.247322
"""Diabetes""",4.887763,7.595284,4.896993,7.37063
"""Leber's hereditary optic neuro…",4.883083,7.529758,4.907231,7.474441
"""Tay-Sachs""",4.900089,7.647843,4.902508,7.416985


In [20]:
df_inner.with_columns(
    diferencia_red = (pl.col("mean_red_m")- pl.col("mean_red_h")).abs(),
    diferencia_white = (pl.col("mean_white_m") - pl.col("mean_white_h")).abs())

Disorder Subclass,mean_red_m,mean_white_m,mean_red_h,mean_white_h,diferencia_red,diferencia_white
str,f64,f64,f64,f64,f64,f64
"""Hemochromatosis""",4.889836,7.454434,4.892835,7.457545,0.002999,0.003111
"""Cancer""",4.883708,7.661741,4.851911,7.765564,0.031798,0.103823
"""Mitochondrial myopathy""",4.893421,7.462122,4.897351,7.519142,0.003929,0.05702
"""Cystic fibrosis""",4.898628,7.630248,4.902574,7.468543,0.003946,0.161705
"""Leigh syndrome""",4.896911,7.419381,4.905123,7.450601,0.008212,0.03122
"""Alzheimer's""",4.896252,7.740429,4.897511,7.247322,0.001259,0.493107
"""Diabetes""",4.887763,7.595284,4.896993,7.37063,0.00923,0.224654
"""Leber's hereditary optic neuro…",4.883083,7.529758,4.907231,7.474441,0.024147,0.055317
"""Tay-Sachs""",4.900089,7.647843,4.902508,7.416985,0.002419,0.230857


In [21]:
## Agrega la información de este DataFrame al DataFrame original. Además, agrega una nueva columna que indique si el instituto tieen
# algún nivel de especialización (asume que los que no están en la lista no tienen especialización). 
# ¿Qué tipo de join has usado y por qué? Si quisiéramos conservar sólo los registros de pacientes que tienen un instituto especializado, ¿qué tipo de join usarías?

In [23]:
df_inst = pl.DataFrame(
    {
        "Institute Name": [
            "Dana-farber Cancer Institute",
            "Beth Israel Deaconess Medical Center",
            "New England Medical Center",
        ],
        "specialization_level": ["High", "Medium", "High"],
    }
)

In [27]:
df_join_inst = df_datos.join(df_inst, on = "Institute Name", how="left")
df_join_inst

Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,specialization_level
str,f64,str,str,str,str,f64,str,str,str,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,str,str,str
"""PID0x6418""",2.0,"""Yes""","""No""","""Yes""","""No""",4.760603,"""Richard""",null,"""Larre""",null,null,"""Boston Specialty & Rehabilitat…","""55 FRUIT ST CENTRAL, MA 02114 …","""Alive""","""Normal (30-60)""","""Normal""",0.0,null,null,1.0,0.0,"""Yes""","""High""",null,null,"""Not applicable""","""Institute""","""No""",null,"""No""","""No""","""No""","""Yes""",null,null,9.857562,null,1.0,1.0,1.0,1.0,1.0,"""Mitochondrial genetic inherita…","""Leber's hereditary optic neuro…",null
"""PID0x25d5""",4.0,"""Yes""","""Yes""","""No""","""No""",4.910669,"""Mike""",null,"""Brycen""",null,23.0,"""St. Margaret's Hospital For Wo…","""1515 COMMONWEALTH AV ALLSTON/B…","""Deceased""","""Tachypnea""","""Normal""",null,0.0,0.0,1.0,0.0,"""Yes""","""High""",null,"""No""","""None""",null,"""Yes""","""Yes""","""Not applicable""","""Not applicable""","""No""","""Yes""",null,"""Multiple""",5.52256,"""normal""",1.0,null,1.0,1.0,0.0,null,"""Cystic fibrosis""",null
"""PID0x4a82""",6.0,"""Yes""","""No""","""No""","""No""",4.893297,"""Kimberly""",null,"""Nashon""",41.0,22.0,null,"""-""","""Alive""","""Normal (30-60)""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""Low""",null,"""No record""","""Not applicable""",null,"""Yes""","""No""","""Yes""",null,"""Yes""","""Yes""",4.0,"""Singular""",null,"""normal""",0.0,1.0,1.0,1.0,1.0,"""Multifactorial genetic inherit…","""Diabetes""",null
"""PID0x4ac8""",12.0,"""Yes""","""No""","""Yes""","""No""",4.70528,"""Jeffery""","""Hoelscher""","""Aayaan""",21.0,null,null,"""55 FRUIT ST CENTRAL, MA 02114 …","""Deceased""","""Tachypnea""","""Normal""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""High""","""Male""","""Not available""","""No""","""Institute""","""No""","""Yes""","""-""","""Not applicable""",null,"""Yes""",1.0,"""Singular""",7.919321,"""inconclusive""",0.0,0.0,1.0,0.0,0.0,"""Mitochondrial genetic inherita…","""Leigh syndrome""",null
"""PID0x1bf7""",11.0,"""Yes""","""No""",null,"""Yes""",4.720703,"""Johanna""","""Stutzman""","""Suave""",32.0,null,"""Carney Hospital""","""300 LONGWOOD AV FENWAY/KENMORE…","""Alive""","""Tachypnea""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,null,"""Low""","""Male""","""Not available""","""Not applicable""","""Institute""","""No""","""Yes""","""-""","""Not applicable""","""Yes""","""No""",4.0,"""Multiple""",4.09821,null,0.0,0.0,0.0,0.0,null,"""Multifactorial genetic inherit…","""Cancer""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PID0x5598""",4.0,"""Yes""","""Yes""","""Yes""","""No""",5.258298,"""Lynn""",null,"""Alhassane""",35.0,64.0,"""Franciscan Children's Hospita…","""1153 CENTRE ST JAMAICA PLAIN, …","""Deceased""","""Normal (30-60)""","""Tachycardia""",null,0.0,null,1.0,0.0,"""Yes""","""High""","""Female""","""No""","""No""","""Institute""",null,"""No""","""Not applicable""","""No""","""Yes""","""No""",3.0,"""Multiple""",6.584811,"""inconclusive""",0.0,0.0,1.0,0.0,0.0,"""Mitochondrial genetic inherita…","""Leigh syndrome""",null


In [29]:
df_join_inst = df_join_inst.with_columns(
    pl.col("specialization_level").is_not_null().alias("has_specialization")
)

df_join_inst

Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,specialization_level,has_specialization
str,f64,str,str,str,str,f64,str,str,str,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,str,str,str,bool
"""PID0x6418""",2.0,"""Yes""","""No""","""Yes""","""No""",4.760603,"""Richard""",null,"""Larre""",null,null,"""Boston Specialty & Rehabilitat…","""55 FRUIT ST CENTRAL, MA 02114 …","""Alive""","""Normal (30-60)""","""Normal""",0.0,null,null,1.0,0.0,"""Yes""","""High""",null,null,"""Not applicable""","""Institute""","""No""",null,"""No""","""No""","""No""","""Yes""",null,null,9.857562,null,1.0,1.0,1.0,1.0,1.0,"""Mitochondrial genetic inherita…","""Leber's hereditary optic neuro…",null,false
"""PID0x25d5""",4.0,"""Yes""","""Yes""","""No""","""No""",4.910669,"""Mike""",null,"""Brycen""",null,23.0,"""St. Margaret's Hospital For Wo…","""1515 COMMONWEALTH AV ALLSTON/B…","""Deceased""","""Tachypnea""","""Normal""",null,0.0,0.0,1.0,0.0,"""Yes""","""High""",null,"""No""","""None""",null,"""Yes""","""Yes""","""Not applicable""","""Not applicable""","""No""","""Yes""",null,"""Multiple""",5.52256,"""normal""",1.0,null,1.0,1.0,0.0,null,"""Cystic fibrosis""",null,false
"""PID0x4a82""",6.0,"""Yes""","""No""","""No""","""No""",4.893297,"""Kimberly""",null,"""Nashon""",41.0,22.0,null,"""-""","""Alive""","""Normal (30-60)""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""Low""",null,"""No record""","""Not applicable""",null,"""Yes""","""No""","""Yes""",null,"""Yes""","""Yes""",4.0,"""Singular""",null,"""normal""",0.0,1.0,1.0,1.0,1.0,"""Multifactorial genetic inherit…","""Diabetes""",null,false
"""PID0x4ac8""",12.0,"""Yes""","""No""","""Yes""","""No""",4.70528,"""Jeffery""","""Hoelscher""","""Aayaan""",21.0,null,null,"""55 FRUIT ST CENTRAL, MA 02114 …","""Deceased""","""Tachypnea""","""Normal""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""High""","""Male""","""Not available""","""No""","""Institute""","""No""","""Yes""","""-""","""Not applicable""",null,"""Yes""",1.0,"""Singular""",7.919321,"""inconclusive""",0.0,0.0,1.0,0.0,0.0,"""Mitochondrial genetic inherita…","""Leigh syndrome""",null,false
"""PID0x1bf7""",11.0,"""Yes""","""No""",null,"""Yes""",4.720703,"""Johanna""","""Stutzman""","""Suave""",32.0,null,"""Carney Hospital""","""300 LONGWOOD AV FENWAY/KENMORE…","""Alive""","""Tachypnea""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,null,"""Low""","""Male""","""Not available""","""Not applicable""","""Institute""","""No""","""Yes""","""-""","""Not applicable""","""Yes""","""No""",4.0,"""Multiple""",4.09821,null,0.0,0.0,0.0,0.0,null,"""Multifactorial genetic inherit…","""Cancer""",null,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PID0x5598""",4.0,"""Yes""","""Yes""","""Yes""","""No""",5.258298,"""Lynn""",null,"""Alhassane""",35.0,64.0,"""Franciscan Children's Hospita…","""1153 CENTRE ST JAMAICA PLAIN, …","""Deceased""","""Normal (30-60)""","""Tachycardia""",null,0.0,null,1.0,0.0,"""Yes""","""High""","""Female""","""No""","""No""","""Institute""",null,"""No""","""Not applicable""","""No""","""Yes""","""No""",3.0,"""Multiple""",6.584811,"""inconclusive""",0.0,0.0,1.0,0.0,0.0,"""Mit

1. Crea una tabla de resumen en formato “ancho” que muestre la edad media de los pacientes por desorden genético y género, presentando los desordenes genéticos como filas y los géneros como columnas.

In [32]:
tabla_resumen= (df_datos.group_by("Genetic Disorder", "Gender").agg(
                media= pl.col("Patient Age").mean()
).pivot(
    index="Genetic Disorder",
    on= "Gender",
    values="media"
)
    
)

tabla_resumen

Genetic Disorder,null,Female,Ambiguous,Male
str,f64,f64,f64,f64
"""Mitochondrial genetic inherita…",6.865672,6.928251,7.074872,6.946124
"""Single-gene inheritance diseas…",6.983103,7.038767,6.959889,6.988362
"""Multifactorial genetic inherit…",6.677419,6.832155,6.723101,7.222973
null,7.334459,6.8829,6.953819,6.90128


In [ ]:
#"Crea una tabla de resumen en formato “largo” que muestre por cada paciente las edades del paciente, de la madre y del padre. 
#Las columnas deben ser: patient_id, type (con valores “patient”, “mother”, “father”) y age. Descarta los registros que no tengan edad informada.

In [33]:
df_datos.columns

['Patient Id',
 'Patient Age',
 "Genes in mother's side",
 'Inherited from father',
 'Maternal gene',
 'Paternal gene',
 'Blood cell count (mcL)',
 'Patient First Name',
 'Family Name',
 "Father's name",
 "Mother's age",
 "Father's age",
 'Institute Name',
 'Location of Institute',
 'Status',
 'Respiratory Rate (breaths/min)',
 'Heart Rate (rates/min',
 'Test 1',
 'Test 2',
 'Test 3',
 'Test 4',
 'Test 5',
 'Parental consent',
 'Follow-up',
 'Gender',
 'Birth asphyxia',
 'Autopsy shows birth defect (if applicable)',
 'Place of birth',
 'Folic acid details (peri-conceptional)',
 'H/O serious maternal illness',
 'H/O radiation exposure (x-ray)',
 'H/O substance abuse',
 'Assisted conception IVF/ART',
 'History of anomalies in previous pregnancies',
 'No. of previous abortion',
 'Birth defects',
 'White Blood cell count (thousand per microliter)',
 'Blood test result',
 'Symptom 1',
 'Symptom 2',
 'Symptom 3',
 'Symptom 4',
 'Symptom 5',
 'Genetic Disorder',
 'Disorder Subclass']

In [37]:
df_datos.select('Patient Id','Patient Age',"Mother's age","Father's age")

Patient Id,Patient Age,Mother's age,Father's age
str,f64,f64,f64
"""PID0x6418""",2.0,null,null
"""PID0x25d5""",4.0,null,23.0
"""PID0x4a82""",6.0,41.0,22.0
"""PID0x4ac8""",12.0,21.0,null
"""PID0x1bf7""",11.0,32.0,null
…,…,…,…
"""PID0x5598""",4.0,35.0,64.0
"""PID0x19cb""",8.0,null,56.0
"""PID0x3c4f""",8.0,35.0,51.0


In [40]:
df_datos.select('Patient Id',patient='Patient Age',mother="Mother's age",father="Father's age").unpivot(index="Patient Id",variable_name="type",value_name="age").drop_nulls("age")


Patient Id,type,age
str,str,f64
"""PID0x6418""","""patient""",2.0
"""PID0x25d5""","""patient""",4.0
"""PID0x4a82""","""patient""",6.0
"""PID0x4ac8""","""patient""",12.0
"""PID0x1bf7""","""patient""",11.0
…,…,…
"""PID0x26b4""","""father""",50.0
"""PID0x5598""","""father""",64.0
"""PID0x19cb""","""father""",56.0
